# PCA intuition

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.gridspec as gridspec
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

np.random.seed(42)

# Generate data
num_points = 30
n_frames_per_angle = 4 # how many frames per angle. The lower - the faster.
X = np.random.randn(num_points, 2)
X = X @ np.linalg.cholesky(np.array([[1, 0.6], [0.6, 0.6]]))
X = X + np.ones(2)

# Normalize data
center_point = np.mean(X, axis=0)
X_std = X - center_point

# PCA components
_, v = np.linalg.eig(np.cov(X_std, rowvar=False))

# Set up the grid
gs = gridspec.GridSpec(2, 1, height_ratios=[5, 1])  # Two rows, one column, with the first row 3 times the height of the second

fig = plt.figure(figsize=(5, 6))  # Adjust the total figure size as necessary

ax = plt.subplot(gs[0])  # The first subplot
ax2 = plt.subplot(gs[1])  # The second subplot

scatter = ax.scatter(X[:,0], X[:,1], color='b', label="Data")

direction_line, = ax.plot([], [], 'k')
v_main = v.T[0]
ax.plot([-v_main[0]*3 + center_point[0], 
                             v_main[0]*3 + center_point[0]], 
                            [-v_main[1]*3 + center_point[1],
                            v_main[1]*3 + center_point[1]], label="First singular vector of X")

projection_points, = ax.plot([], [], 'ro', markersize=5, label="Projections")
projection_lines = [ax.plot([], [], 'r')[0] for _ in range(num_points)]

direction_line2, = ax2.plot([-3.5, 3.5], [0,0], 'k')
projections, = ax2.plot([],[], 'ro', markersize=7)

def init():
    ax.axis('equal')
    ax.grid(linestyle=":")
    ax.scatter(x=center_point[0], y=center_point[1], c='k')
    ax.legend(loc="upper right")
    ax.set_title("PCA")
    ax.text(1, 1.02, "@fminxyz", transform=ax.transAxes, color='gray', va='baseline', ha='right', fontsize=9)
    
    ax2.set_xlim(-3.5, 3.5)
    ax2.set_ylim(-1, 1)
    w = np.array([0, 0])
    ax2.grid(linestyle=":")
    ax2.set_title("Projections on the First Principal Component\n"
                  f"Variance of the projections: {np.linalg.norm(X_std@w)**2:.1f}")
    fig.tight_layout()
    return scatter, direction_line, projection_points, projection_lines


def update(frame):
    ax.set_xlim(-3.5+center_point[0], 3.5+center_point[0])
    ax.set_ylim(-3.5+center_point[1], 3.5+center_point[1])
    alpha = frame/n_frames_per_angle
    w = np.array([np.cos(np.radians(alpha)), np.sin(np.radians(alpha))])
    z = X_std @ w.reshape(-1, 1) @ w.reshape(1, -1) + center_point

    for i in range(num_points):
        projection_lines[i].set_data([X[i, 0], z[i, 0]], [X[i, 1], z[i, 1]])
        projection_lines[i].set_color('r')
    
    projection_points.set_data(z[:, 0], z[:, 1])
    # distances = pdist(z)
    # max_distance = np.max(distances)
    # projection_points.set_label(f"Max Distance: {max_distance:.2f}")
    
    direction_line.set_data([-w[0]*3 + center_point[0], 
                             w[0]*3 + center_point[0]], 
                            [-w[1]*3 + center_point[1],
                            w[1]*3 + center_point[1]])

    ax2.set_xlim(-3.5, 3.5)
    ax2.set_ylim(-1, 1)
    projections.set_data(X_std@w, np.zeros(len(X_std@w)))
    ax2.set_title("Projections on the First Principal Component\n"
                  f"Variance of the projections: {np.linalg.norm(X_std@w)**2:.1f}")

    return direction_line, projection_points, projection_lines

ani = animation.FuncAnimation(fig, update, 
                              frames=np.arange(0, n_frames_per_angle*180), 
                              interval=1000/60, # 60 fps
                              init_func=init)

plt.close()

from IPython import display
html = display.HTML(ani.to_html5_video())
display.display(html)

# # Uncomment to save to the file
# ani.save("PCA_animation.mp4", writer='ffmpeg', fps=60, dpi=300)

/var/folders/7m/3rbdnx5n5sz625f3l87m91cc0000gn/T/ipykernel_72727/3281343646.py:61: UserWarning:

The figure layout has changed to tight



# PCA fails for data distinguishing

In [35]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.gridspec as gridspec
from matplotlib.legend_handler import HandlerTuple

np.random.seed(4)

# Generate data
num_points = 40
n_frames_per_angle = 4 # how many frames per angle. The lower - the faster.
X_1 = np.random.randn(int(num_points/2), 2)
X_1 = X_1 @ np.array([[1.1, 0.0], [0.0, 0.2]])

X_2 = np.random.randn(int(num_points/2), 2)
X_2 = X_2 @ np.array([[1.1, 0.0], [0.0, 0.2]])
X_2 = X_2 + np.array([0, 2])

X = np.vstack([X_1, X_2])

# Normalize data
center_point = np.mean(X, axis=0)
X_std = X - center_point

# PCA components
_, v = np.linalg.eig(X_std.T @ X_std)
v_main = v.T[0]

# Set up the grid
gs = gridspec.GridSpec(2, 1, height_ratios=[5, 1])  # Two rows, one column, with the first row 3 times the height of the second

fig = plt.figure(figsize=(5, 6))  # Adjust the total figure size as necessary

ax = plt.subplot(gs[0])  # The first subplot
ax2 = plt.subplot(gs[1])  # The second subplot

# Split into two separate scatter plots
scatter_1 = ax.scatter(X_1[:,0], X_1[:,1], color='b', marker='x')
scatter_2 = ax.scatter(X_2[:,0], X_2[:,1], color='b', marker='o')

direction_line, = ax.plot([], [], 'k')
eig_plot, = ax.plot([-v_main[0]*3 + center_point[0], 
                             v_main[0]*3 + center_point[0]], 
                            [-v_main[1]*3 + center_point[1],
                            v_main[1]*3 + center_point[1]])

projection_points1, = ax.plot([], [], 'ro', markersize=5)
projection_points2, = ax.plot([], [], 'rx', markersize=5)
projection_lines = [ax.plot([], [], 'r')[0] for _ in range(num_points)]

direction_line2, = ax2.plot([-3.5, 3.5], [0,0], 'k')
projections1, = ax2.plot([],[], 'ro', markersize=7)
projections2, = ax2.plot([],[], 'rx', markersize=7)

def init():
    ax.axis('equal')
    ax.grid(linestyle=":")
    ax.scatter(x=center_point[0], y=center_point[1], c='k')
    ax.legend([(scatter_1, scatter_2), (projection_points2, projection_points1), eig_plot], ['Data',"Projections", 'First eigenvector of X'],
               handler_map={tuple: HandlerTuple(ndivide=None)}, loc="upper right")
    ax.set_title("PCA")
    ax.text(1, 1.02, "@fminxyz", transform=ax.transAxes, color='gray', va='baseline', ha='right', fontsize=9)
    
    ax2.set_xlim(-3.5, 3.5)
    ax2.set_ylim(-1, 1)
    w = np.array([0, 0])
    ax2.grid(linestyle=":")
    ax2.set_title("Projections on the First Principal Component\n"
                  f"Variance of the projections: {np.linalg.norm(X_std@w)**2:.1f}")
    fig.tight_layout()
    return scatter, direction_line, projection_points1, projection_points2, projection_lines


def update(frame):
    ax.set_xlim(-3.5+center_point[0], 3.5+center_point[0])
    ax.set_ylim(-3.5+center_point[1], 3.5+center_point[1])
    alpha = frame/n_frames_per_angle
    w = np.array([np.cos(np.radians(alpha)), np.sin(np.radians(alpha))])
    z = X_std @ w.reshape(-1, 1) @ w.reshape(1, -1) + center_point

    for i in range(num_points):
        projection_lines[i].set_data([X[i, 0], z[i, 0]], [X[i, 1], z[i, 1]])
        projection_lines[i].set_color('r')
    
    projection_points1.set_data(z[int(num_points/2):num_points, 0], z[int(num_points/2):num_points, 1])
    projection_points2.set_data(z[:int(num_points/2), 0], z[:int(num_points/2), 1])
    
    direction_line.set_data([-w[0]*3 + center_point[0], 
                             w[0]*3 + center_point[0]], 
                            [-w[1]*3 + center_point[1],
                            w[1]*3 + center_point[1]])

    ax2.set_xlim(-3.5, 3.5)
    ax2.set_ylim(-1, 1)
    projections1.set_data(X_std[int(num_points/2):num_points]@w, np.zeros(len(X_std[int(num_points/2):num_points]@w)))
    projections2.set_data(X_std[:int(num_points/2)]@w, np.zeros(len(X_std[:int(num_points/2)]@w)))
    ax2.set_title("Projections on the First Principal Component\n"
                  f"Variance of the projections: {np.linalg.norm(X_std@w)**2:.1f}")

    return direction_line, projection_points1, projection_points2, projection_lines

ani = animation.FuncAnimation(fig, update, 
                              frames=np.arange(0, n_frames_per_angle*180), 
                              interval=1000/60, # 60 fps
                              init_func=init)

plt.close()
from IPython import display
html = display.HTML(ani.to_html5_video())
display.display(html)

# # Uncomment to save to the file
ani.save("PCA_animation.mp4", writer='ffmpeg', fps=60, dpi=300)

/var/folders/7m/3rbdnx5n5sz625f3l87m91cc0000gn/T/ipykernel_72727/822554100.py:70: UserWarning:

The figure layout has changed to tight

